In [2]:
# --- Configuration ---
BASE = "https://api.thucchien.ai"                  # BTC gateway root
API = f"{BASE}/chat/completions"                   # OpenAI-compatible route
KEY = "sk-qO0UQ8AWiW5BFklm7xJ36g"               # <-- paste your BTC key here

import os, json, re, pathlib, shutil, requests, textwrap, time
assert KEY.startswith("sk-"), "Please set your BTC key (looks like 'sk-...')"

print("Using endpoint:", API)
print("KEY prefix:", KEY[:6] + "…")


Using endpoint: https://api.thucchien.ai/chat/completions
KEY prefix: sk-qO0…


In [9]:
def chat_completions(messages, model="gemini-2.5-flash", max_tokens=4000, temperature=0.6):
    """Call BTC gateway /chat/completions and force plain-text output (no hidden reasoning)."""
    headers = {
        "Authorization": f"Bearer {KEY}",
        "Content-Type": "application/json",
        "x-litellm-model-strict": "true",    # ask proxy to not fallback
    }
    payload = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "disable_fallbacks": True,           # no auto model downgrade
        "response_format": {"type": "text"}, # prefer strict plain text
        "extra_body": {"response_mime_type": "text/plain"}
    }
    r = requests.post(API, headers=headers, json=payload, timeout=180)
    if r.status_code >= 400:
        print("Status:", r.status_code)
        try:
            print("Body:", json.dumps(r.json(), ensure_ascii=False, indent=2)[:1200])
        except Exception:
            print("Body (raw):", r.text[:1200])
        r.raise_for_status()
    data = r.json()
    # Warn if proxy still changed the model:
    resp_model = data.get("model")
    if resp_model and resp_model != model:
        print(f"⚠️ Proxy responded with model={resp_model} (requested {model}).")
    return data

def get_text_from_chat_response(data: dict) -> str:
    try:
        return data["choices"][0]["message"]["content"].strip()
    except Exception:
        return json.dumps(data, ensure_ascii=False, indent=2)


In [4]:
def extract_project_files(block_text: str, out_dir: str):
    """Parse BEGIN_PROJECT bundle and write files into out_dir."""
    out = pathlib.Path(out_dir)
    if out.exists():
        shutil.rmtree(out)
    out.mkdir(parents=True, exist_ok=True)

    lines = block_text.splitlines()
    in_proj = False
    capturing = False
    current_file = None
    buf = []

    def flush_file(path, content_lines):
        p = out / path.lstrip("/")
        p.parent.mkdir(parents=True, exist_ok=True)
        p.write_text("\n".join(content_lines), encoding="utf-8")

    i = 0
    while i < len(lines):
        line = lines[i]
        if line.strip() == "BEGIN_PROJECT":
            in_proj = True; i += 1; continue
        if in_proj and line.startswith("FILE: "):
            current_file = line[6:].strip()
            i += 1
            if i < len(lines) and lines[i].strip() == "<<<":
                capturing = True; buf = []; i += 1; continue
            else:
                raise ValueError("Expected <<< after FILE header")
        if in_proj and capturing:
            if line.strip() == ">>>":
                flush_file(current_file, buf)
                capturing = False; current_file = None; i += 1; continue
            else:
                buf.append(line); i += 1; continue
        if in_proj and line.strip() == "END_PROJECT":
            break
        i += 1

    tree = [str(p.relative_to(out)) for p in sorted(out.rglob("*"))]
    return out_dir, tree


In [5]:
outline_messages = [
    {"role":"system","content":"Bạn là chuyên gia lập trình web. Trả lời Markdown ngắn gọn, không reasoning."},
    {"role":"user","content":"""
     Sản phẩm: Xây dựng một website quảng bá du lịch cho địa điểm bên dưới.
Địa điểm cụ thể: Bản Yên Hoà, Xã Mỹ Lý, Tỉnh Nghệ An.
Đối tượng website: Khách du lịch có nhu cầu tìm hiểu kỹ thông tin trước khi quyết định hành trình.
Nền tảng: Hỗ trợ Web desktop.
Yêu cầu chính: Website cần cung cấp thông tin đầy đủ, tin cậy, có bố cục logic, giao diện thẩm mỹ và thân thiện với người dùng (user-friendly).
Yêu cầu về tính sáng tạo: Sản phẩm phải là sáng tạo mới hoàn toàn, đáp ứng đầy đủ yêu cầu của đề bài, tuân thủ các quy định pháp lý và chuẩn mực giáo dục. Không sao chép hoặc mô phỏng từ bất kỳ sản phẩm nào có sẵn.
Ngôn ngữ: Tiếng Việt.

     """}
]
resp_o = chat_completions(outline_messages, model="gemini-2.5-pro", max_tokens=800, temperature=0.5)
outline_md = get_text_from_chat_response(resp_o)

from pathlib import Path
p = Path("website_outline.md")
p.write_text(outline_md, encoding="utf-8")
print("Saved:", str(p))
print(outline_md[:800])


Saved: website_outline.md
{
  "id": "1Nj6aOjjNIuU1e8PiK-swA4",
  "created": 1761269963,
  "model": "gemini-2.5-pro",
  "object": "chat.completion",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "role": "assistant"
      }
    }
  ],
  "usage": {
    "completion_tokens": 799,
    "prompt_tokens": 208,
    "total_tokens": 1007,
    "completion_tokens_details": {
      "reasoning_tokens": 799,
      "text_tokens": 0
    },
    "prompt_tokens_details": {
      "text_tokens": 208
    }
  },
  "vertex_ai_grounding_metadata": [],
  "vertex_ai_url_context_metadata": [],
  "vertex_ai_safety_results": [],
  "vertex_ai_citation_metadata": []
}


In [ ]:
react_prompt = f"""
Bạn là Website Project Generator. Hãy tạo **skeleton React + Vite + Tailwind** (không cài dependency, chỉ sinh file).

[Thông tin dự án]
1. Cấu trúc Website (Sitemap)
Trang Chủ (Homepage)

Hero Section: Video/slideshow toàn cảnh Yên Hoà.
Giới thiệu ngắn gọn: "Yên Hoà - Vẻ đẹp tiềm ẩn nơi đại ngàn".
Các trải nghiệm nổi bật (icon + mô tả ngắn): Trekking, Chèo thuyền, Giao lưu văn hoá, Thưởng thức ẩm thực.
Lịch trình gợi ý (3 lựa chọn chính).
Góc nhìn du khách (Testimonials).
Kêu gọi hành động (CTA): "Lên kế hoạch cho chuyến đi của bạn".
Khám Phá Yên Hoà

Vẻ Đẹp Thiên Nhiên: Sông Nậm Nơn, rừng nguyên sinh, thác nước, hệ động thực vật.
Văn Hóa Bản Địa: Đời sống người Thái, kiến trúc nhà sàn, trang phục, lễ hội, nghề thủ công (dệt vải).
Ẩm Thực Đặc Sản: Cá nướng (Pa Pỉnh Tộp), Cơm lam, Rau rừng, Rượu cần.
Trải Nghiệm Độc Đáo: Danh sách các hoạt động chi tiết (trekking, chèo thuyền kayak, tắm suối, học dệt vải, giao lưu văn nghệ).
Lên Kế Hoạch

Lưu Trú: Danh sách các homestay, thông tin chi tiết, hình ảnh, tiện nghi và liên hệ đặt phòng.
Di Chuyển: Hướng dẫn đường đi từ Hà Nội/Vinh, phương tiện (xe khách, xe máy), bản đồ.
Lịch Trình Gợi Ý:
2 ngày 1 đêm: "Chạm ngõ Yên Hoà".
3 ngày 2 đêm: "Sống chậm cùng Yên Hoà".
Lưu ý khi du lịch: Thời điểm lý tưởng, vật dụng cần chuẩn bị, quy tắc ứng xử văn hoá.
Thư Viện

Hình ảnh.
Video & Flycam.
Blog: Chuyện của Yên Hoà

Các bài viết sâu về văn hoá, con người, câu chuyện du lịch.
Liên Hệ

Form liên hệ tư vấn.
Thông tin hotline, email.
Bản đồ Google Maps nhúng.
2. Công nghệ & Nền tảng
Frontend: Next.js (React) - Tối ưu SEO, tốc độ tải trang nhanh.
Backend: Headless CMS (Strapi, Sanity) - Dễ dàng quản lý nội dung cho người không chuyên về kỹ thuật.
Database: PostgreSQL.
Hosting: Vercel hoặc Netlify.
Map API: Mapbox hoặc Google Maps API.
3. Thiết kế Giao diện & Trải nghiệm Người dùng (UI/UX)
Phong cách: Tối giản, mộc mạc, hiện đại.
Màu sắc chủ đạo: Xanh lá cây (của rừng), Nâu (của gỗ, đất), Trắng/Be (tạo không gian thoáng).
Typography: Font serif (Playfair Display) cho tiêu đề, sans-serif (Montserrat) cho nội dung.
Điểm nhấn: Sử dụng họa tiết thổ cẩm của người Thái làm đường viền (border) hoặc nền (background pattern) một cách tinh tế.
Tương tác: Hiệu ứng cuộn trang parallax, micro-interactions nhẹ nhàng.
4. Yếu tố Sáng tạo & Độc đáo
Bản đồ trải nghiệm tương tác: Một bản đồ vẽ tay (digital art) của Bản Yên Hoà, người dùng có thể click vào các địa điểm (thác, nhà sàn, bến thuyền) để xem popup thông tin và hình ảnh nhanh.
Podcast "Âm thanh từ đại ngàn": Tích hợp các file audio ngắn ghi lại âm thanh của Yên Hoà (tiếng suối chảy, tiếng chim hót, tiếng khèn).
Tính năng "Tạo lịch trình của tôi": Cho phép người dùng kéo-thả các hoạt động yêu thích vào một timeline để tự xây dựng lịch trình cá nhân và có thể in ra hoặc lưu lại.
Booking trực tiếp: Kết nối trực tiếp du khách với các chủ homestay qua form đặt phòng đơn giản, minh bạch, hỗ trợ du lịch cộng đồng.

[Định dạng đầu ra BẮT BUỘC] — chỉ xuất đúng khối sau:
BEGIN_PROJECT
FILE: /index.html
<<<
(nội dung file)
>>>
FILE: /src/main.jsx
<<<
(nội dung file)
>>>
FILE: /src/App.jsx
<<<
(nội dung file)
>>>
FILE: /src/components/Hero.jsx
<<<
(nội dung file)
>>>
FILE: /src/components/Timeline.jsx
<<<
(nội dung file)
>>>
FILE: /src/components/Gallery.jsx
<<<
(nội dung file)
>>>
FILE: /src/components/About.jsx
<<<
(nội dung file)
>>>
FILE: /src/components/Contact.jsx
<<<
(nội dung file)
>>>
FILE: /src/index.css
<<<
(nội dung file)
>>>
FILE: /tailwind.config.cjs
<<<
(nội dung file)
>>>
FILE: /postcss.config.cjs
<<<
(nội dung file)
>>>
FILE: /package.json
<<<
(nội dung file; scripts dạng vite: dev/build/preview)
>>>
FILE: /AI_GENERATION_NOTES.md
<<<
- Model: gemini-2.5-pro (BTC gateway, /chat/completions)
- Prompt: React Vite skeleton
- Ngày tạo: {time.strftime("%Y-%m-%d %H:%M:%S")}
>>>
END_PROJECT
"""
messages_react = [
    {"role": "system", "content": "Bạn là Website Project Generator. Chỉ trả về đúng khối BEGIN_PROJECT/FILE/<<</>>>/END_PROJECT, không thêm giải thích."},
    {"role": "user", "content": react_prompt}
]

resp_r = chat_completions(messages_react, model="gemini-2.5-pro", max_tokens=7000, temperature=0.6)
text_block_r = get_text_from_chat_response(resp_r)
print(text_block_r[:1200])


BEGIN_PROJECT
FILE: /index.html
<<<
<!doctype html>
<html lang="vi">
  <head>
    <meta charset="UTF-8" />
    <link rel="icon" type="image/svg+xml" href="/vite.svg" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Yên Hoà - Vẻ đẹp tiềm ẩn nơi đại ngàn</title>
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Montserrat:wght@400;700&family=Playfair+Display:wght@700&display=swap" rel="stylesheet">
  </head>
  <body>
    <div id="root"></div>
    <script type="module" src="/src/main.jsx"></script>
  </body>
</html>
>>>
FILE: /src/main.jsx
<<<
import React from 'react'
import ReactDOM from 'react-dom/client'
import App from './App.jsx'
import './index.css'

ReactDOM.createRoot(document.getElementById('root')).render(
  <React.StrictMode>
    <App />
  </React.StrictMode>,
)
>>>
FILE: /src/App.jsx
<<<
impor

In [7]:
out_dir_r, tree_r = extract_project_files(text_block_r, out_dir="website_react_output")
print("Written to:", out_dir_r)
print("\n".join(tree_r[:60]))


Written to: website_react_output
AI_GENERATION_NOTES.md
index.html
package.json
postcss.config.cjs
src
src\App.jsx
src\components
src\components\About.jsx
src\components\Contact.jsx
src\components\Gallery.jsx
src\components\Hero.jsx
src\components\Timeline.jsx
src\index.css
src\main.jsx
tailwind.config.cjs


In [10]:
outline_messages = [
    {"role":"system","content":"Bạn là chuyên gia lập trình web."},
    {"role":"user","content":"[plugin:vite:css] [postcss] It looks like you're trying to use `tailwindcss` directly as a PostCSS plugin. The PostCSS plugin has moved to a separate package, so to continue using Tailwind CSS with PostCSS you'll need to install `@tailwindcss/postcss` and update your PostCSS configuration"}
]
resp_o = chat_completions(outline_messages, model="gemini-2.5-flash", max_tokens=800, temperature=0.5)
outline_md = get_text_from_chat_response(resp_o)

from pathlib import Path
p = Path("huongdan.txt")
p.write_text(outline_md, encoding="utf-8")
print("Saved:", str(p))
print(outline_md[:800])


Saved: huongdan.txt
{
  "id": "I9z6aIi5BZLJ1e8P_fnmyAU",
  "created": 1761270814,
  "model": "gemini-2.5-flash",
  "object": "chat.completion",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "role": "assistant"
      }
    }
  ],
  "usage": {
    "completion_tokens": 799,
    "prompt_tokens": 75,
    "total_tokens": 874,
    "completion_tokens_details": {
      "reasoning_tokens": 799,
      "text_tokens": 0
    },
    "prompt_tokens_details": {
      "text_tokens": 75
    }
  },
  "vertex_ai_grounding_metadata": [],
  "vertex_ai_url_context_metadata": [],
  "vertex_ai_safety_results": [],
  "vertex_ai_citation_metadata": []
}


In [12]:
import os, json, base64, mimetypes, requests
def _image_to_base64(image_path: str) -> tuple[str, str]:
    mime = mimetypes.guess_type(image_path)[0] or "image/png"
    with open(image_path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    return b64, mime
base64_res=_image_to_base64(image_path="./download.png")
print(base64_res)  # In ra 100 ký tự đầu tiên của chuỗi base64
 


('...', 'image/png')
